<a href="https://colab.research.google.com/github/Eunan02/Premier-league-Predicting-Model/blob/main/DataScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
standings_url="https://fbref.com/en/comps/9/Premier-League-Stats"
data= requests.get(standings_url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(data.text)
standings_table=soup.select('table.stats_table')[0]
links=standings_table.find_all('a')
links=[l.get("href") for l in links]
links=[l for l in links if '/squads/' in l]

team_urls=[f"https://fbref.com{l}" for l in links]
team_urls

team_url=team_urls[0]
data=requests.get(team_url)
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")
matches[0]

soup=BeautifulSoup(data.text)
links=soup.find_all('a')
links=[l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]
data=requests.get(f"https://fbref.com{links[0]}")
shooting=pd.read_html(data.text,match="Shooting")[0]
shooting.head()
shooting.columns=shooting.columns.droplevel()
shooting.head()
shooting["Date"]
team_data=matches[0].merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]], on="Date")

In [ ]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,4-2-3-1,Anthony Taylor,Match Report,NaN,10,2,15.0,1,0,0
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,4-2-3-1,Darren England,Match Report,NaN,19,7,12.6,0,0,0
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,4-2-3-1,Craig Pawson,Match Report,NaN,14,6,14.6,0,0,0
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,4-2-3-1,Jarred Gillett,Match Report,NaN,23,8,16.6,1,0,0
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,4-2-3-1,Robert Jones,Match Report,NaN,22,8,15.3,1,0,0


In [ ]:
import time
years=list(range(2023,2019,-1))
years
all_matches=[]
standingss_url="https://fbref.com/en/comps/9/Premier-League-Stats"
for year in years:
  data=requests.get(standings_url)
  soup=BeautifulSoup(data.text)
  standings_table=soup.select('table.stats_table')[0]

  links = [l.get("href") for l in standings_table.find_all('a')]
  links = [l for l in links if '/squads/' in l]
  team_urls=[f"https://fbref.com{l}" for l in links]

  previous_season= soup.select("a.prev")[0].get("href")
  standings_url=f"https://fbref.com/{previous_season}"

  for team_url in team_urls:
    team_name=team_url.split("/")[-1].replace("-Stats","").replace("-"," ")

    data=requests.get(team_url)
    matches=pd.read_html(data.text,match="Scores & Fixtures")[0]


    soup=BeautifulSoup(data.text)
    links= [l.get("href") for l in soup.find_all('a')]
    links= [l for l in links if l and 'all_comps/shooting/' in l]
    data= requests.get(f"https://fbref.com{links[0]}")
    shooting=pd.read_html(data.text, match="Shooting")[0]
    shooting.columns=shooting.columns.droplevel()

    try:
       team_data=matches.merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]], on="Date")
    except ValueError:
      continue
    team_data = team_data[team_data["Comp"] == "Premier League"]
    
    team_data["Season"]=year
    team_data["Team"]=team_name 
    all_matches.append(team_data)
    time.sleep(1)

  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
match_df=pd.concat(all_matches)
match_df.columns=[c.lower() for c in match_df.columns]
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,Match Report,NaN,10.0,2.0,15.0,1.0,0.0,0.0,2023,Arsenal
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,Match Report,NaN,19.0,7.0,12.6,0.0,0.0,0.0,2023,Arsenal
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,Match Report,NaN,14.0,6.0,14.6,0.0,0.0,0.0,2023,Arsenal
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Match Report,NaN,23.0,8.0,16.6,1.0,0.0,0.0,2023,Arsenal
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,Match Report,NaN,22.0,8.0,15.3,1.0,0.0,0.0,2023,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2020-07-07,18:00,Premier League,Matchweek 34,Tue,Away,L,1,2,Watford,...,Match Report,NaN,13.0,3.0,16.1,1.0,0.0,0.0,2020,Norwich City
39,2020-07-11,12:30,Premier League,Matchweek 35,Sat,Home,L,0,4,West Ham,...,Match Report,NaN,11.0,2.0,18.9,1.0,0.0,0.0,2020,Norwich City
40,2020-07-14,20:15,Premier League,Matchweek 36,Tue,Away,L,0,1,Chelsea,...,Match Report,NaN,2.0,0.0,22.4,0.0,0.0,0.0,2020,Norwich City
41,2020-07-18,17:30,Premier League,Matchweek 37,Sat,Home,L,0,2,Burnley,...,Match Report,NaN,6.0,2.0,12.9,0.0,0.0,0.0,2020,Norwich City


In [ ]:
from google.colab import files
match_df.to_csv("matches.csv")
files.download("matches.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>